# Most-stable allocation under distributional constraints
### Kirill Zakharov
2021

In [1]:
from gurobipy import *
import numpy as np
import random

### Introduce quotas and dimensional information

In [207]:
# Defining Sets
n = 25
m = 5

students = [f"st {i}" for i in range(n)]
jobs = [f"comp {i}" for i in range(m)]

#quotas
upperQ = list(np.random.randint(7, 8, size=m))
lowerQ = list(np.random.randint(4, 5, size=m))

In [208]:
upperQ

[7, 7, 7, 7, 7]

In [209]:
#Students' ranks of companies
a_ranks = [random.sample(list(np.arange(m)+1), m) for i in range(n)]

#Companies' scores of students
c_score = np.array([np.random.randint(1, 10, size=n) for j in range(m)]).T

In [169]:
model= Model("Assignment Model")

### Introduce deficiency variables and binary variables

In [170]:
#Defining the Variable
X = {}
for i in range(n):
    for j in range(m):
        X[i,j] = model.addVar(vtype= GRB.BINARY)
        
d = {}
for i in range(n):
    for j in range(m):
        d[i,j] = model.addVar(lb=0.0, ub=float('inf'), vtype= GRB.CONTINUOUS)

### Define objective function as sum of deficiency variables

In [171]:
#Objective Function
# model.setObjective(quicksum(a_ranks[i][j]*X[i,j] for i in range(n) for j in range(m)), GRB.MINIMIZE)
model.setObjective(quicksum(d[i,j] for i in range(n) for j in range(m)), GRB.MINIMIZE)

## First type of constraints

### Define constraints

In [172]:
#Constraint-1
for i in range(n):
     model.addConstr(quicksum(X[i,j] for j in range(m)) <= 1)
#Constraint-2
for j in range(m):
    model.addConstr(quicksum(X[i,j] for i in range(n)) <= upperQ[j])
    
#Constraint-3
for j in range(m):
    model.addConstr(quicksum(X[i,j] for i in range(n)) >= lowerQ[j])        

# for i in range(n):
#     for j in range(m):
#         model.addConstr(quicksum(X[i,k] for k in range(m) if (a_ranks[i][k] <= a_ranks[i][j]))*upperQ[j] \
#                         + quicksum(X[h,j] for h in range(n) if c_score[h][j] > c_score[i][j]) >= upperQ[j])
#Constraint-4
for i in range(n):
    for j in range(m):
        model.addConstr(quicksum(X[i,k] for k in range(m) if (a_ranks[i][k] <= a_ranks[i][j]))*upperQ[j] \
                        + quicksum(X[h,j] for h in range(n) if c_score[h][j] >= c_score[i][j]) + d[i,j] \
                        >= upperQ[j])
#Constraint-5    
for i in range(n):
    for j in range(m):
        model.addConstr(X[i,j] >= 0)

### Solution

In [173]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 285 rows, 250 columns and 2675 nonzeros
Model fingerprint: 0x974b45ac
Variable types: 125 continuous, 125 integer (125 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Found heuristic solution: objective 329.0000000
Presolve removed 126 rows and 1 columns
Presolve time: 0.01s
Presolved: 159 rows, 249 columns, 2544 nonzeros
Variable types: 0 continuous, 249 integer (125 binary)

Root relaxation: objective 0.000000e+00, 132 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    4  329.00000    0.00000   100%     -    0s
H    0     0                       3.0000000    0.

In [174]:
res1 = np.array(model.X[:125]).reshape((n,m))
res1

array([[0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

In [175]:
a_ranksTable = np.array(a_ranks).reshape((n,m))
a_ranksTable

array([[4, 2, 5, 1, 3],
       [3, 4, 1, 2, 5],
       [3, 2, 1, 5, 4],
       [2, 3, 5, 1, 4],
       [2, 5, 4, 3, 1],
       [5, 3, 4, 2, 1],
       [1, 5, 3, 4, 2],
       [4, 1, 2, 3, 5],
       [4, 3, 1, 5, 2],
       [2, 1, 4, 3, 5],
       [2, 1, 3, 5, 4],
       [3, 2, 1, 5, 4],
       [1, 3, 4, 5, 2],
       [3, 1, 5, 4, 2],
       [4, 5, 1, 3, 2],
       [2, 3, 5, 4, 1],
       [5, 2, 4, 1, 3],
       [3, 1, 5, 4, 2],
       [4, 3, 2, 5, 1],
       [2, 3, 4, 1, 5],
       [1, 5, 2, 4, 3],
       [4, 3, 5, 1, 2],
       [5, 2, 4, 3, 1],
       [3, 4, 5, 2, 1],
       [3, 4, 5, 2, 1]])

### Check how many students have  the choice that differs from initial one

In [176]:
checkMatch = [False for i in range(n)]
for i in range(n):
    if np.where(res1[i] == 1)[0][0] == np.where(a_ranksTable[i] == 1)[0][0]:
        checkMatch[i] = True

In [177]:
checkMatch

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [178]:
n - sum(checkMatch)

1

## Second type of constraints

In [210]:
model= Model("Assignment Model")

#Defining the Variable
X = {}
for i in range(n):
    for j in range(m):
        X[i,j] = model.addVar(vtype= GRB.BINARY)
        
d = {}
for i in range(n):
    for j in range(m):
        d[i,j] = model.addVar(vtype= GRB.BINARY)
        
model.setObjective(quicksum(d[i,j] for i in range(n) for j in range(m)), GRB.MINIMIZE)

In [211]:
#Constraint-1
for i in range(n):
     model.addConstr(quicksum(X[i,j] for j in range(m)) <= 1)
#Constraint-2
for j in range(m):
    model.addConstr(quicksum(X[i,j] for i in range(n)) <= upperQ[j])
    
#Constraint-3
for j in range(m):
    model.addConstr(quicksum(X[i,j] for i in range(n)) >= lowerQ[j])        

# for i in range(n):
#     for j in range(m):
#         model.addConstr(quicksum(X[i,k] for k in range(m) if (a_ranks[i][k] <= a_ranks[i][j]))*upperQ[j] \
#                         + quicksum(X[h,j] for h in range(n) if c_score[h][j] > c_score[i][j]) >= upperQ[j])

for i in range(n):
    for j in range(m):
        model.addConstr(quicksum(X[i,k] for k in range(m) if (a_ranks[i][k] <= a_ranks[i][j]))*upperQ[j] \
                        + quicksum(X[h,j] for h in range(n) if c_score[h][j] >= c_score[i][j])\
                        + d[i,j]*upperQ[j] >= upperQ[j])
    
for i in range(n):
    for j in range(m):
        model.addConstr(X[i,j] >= 0)

In [212]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 285 rows, 250 columns and 2657 nonzeros
Model fingerprint: 0xb76cb43d
Variable types: 0 continuous, 250 integer (250 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Found heuristic solution: objective 73.0000000
Presolve removed 135 rows and 10 columns
Presolve time: 0.01s
Presolved: 150 rows, 240 columns, 2389 nonzeros
Variable types: 0 continuous, 240 integer (240 binary)

Root relaxation: objective 0.000000e+00, 63 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   13   73.00000    0.00000   100%     -    0s
H    0     0                       4.0000000    0.000

In [213]:
res2 = np.array(model.X[:125]).reshape((n,m))
res2

array([[-0., -0., -0.,  1., -0.],
       [-0., -0., -0.,  1., -0.],
       [ 1., -0., -0., -0., -0.],
       [-0., -0.,  1.,  0., -0.],
       [-0., -0.,  1., -0., -0.],
       [ 1., -0., -0., -0., -0.],
       [-0., -0.,  1., -0., -0.],
       [-0.,  1., -0., -0., -0.],
       [-0., -0.,  1., -0., -0.],
       [ 1., -0., -0., -0., -0.],
       [-0.,  1., -0., -0., -0.],
       [-0.,  1., -0., -0., -0.],
       [-0.,  1., -0., -0., -0.],
       [-0., -0., -0., -0.,  1.],
       [-0., -0., -0., -0.,  1.],
       [-0.,  1., -0., -0., -0.],
       [-0., -0., -0.,  1., -0.],
       [-0.,  1., -0., -0., -0.],
       [ 1., -0., -0., -0., -0.],
       [-0., -0.,  1., -0., -0.],
       [-0., -0., -0.,  1., -0.],
       [-0., -0.,  1., -0., -0.],
       [-0., -0., -0., -0.,  1.],
       [-0., -0., -0., -0.,  1.],
       [-0.,  1., -0., -0., -0.]])

In [214]:
a_ranksTable2 = np.array(a_ranks).reshape((n,m))
a_ranksTable2

array([[5, 2, 4, 1, 3],
       [3, 5, 2, 1, 4],
       [1, 3, 5, 2, 4],
       [5, 4, 1, 2, 3],
       [2, 3, 1, 5, 4],
       [2, 3, 1, 4, 5],
       [3, 1, 2, 4, 5],
       [4, 1, 3, 5, 2],
       [4, 3, 1, 2, 5],
       [1, 5, 4, 2, 3],
       [2, 1, 5, 4, 3],
       [2, 1, 3, 5, 4],
       [5, 1, 3, 4, 2],
       [4, 5, 3, 2, 1],
       [5, 4, 2, 3, 1],
       [5, 1, 2, 3, 4],
       [3, 5, 2, 1, 4],
       [5, 1, 4, 2, 3],
       [1, 2, 4, 5, 3],
       [2, 3, 1, 5, 4],
       [2, 3, 5, 1, 4],
       [5, 3, 1, 2, 4],
       [5, 4, 3, 1, 2],
       [4, 5, 3, 2, 1],
       [3, 1, 5, 4, 2]])

In [215]:
checkMatch2 = [False for i in range(n)]
for i in range(n):
    if np.where(res2[i] == 1)[0][0] == np.where(a_ranksTable2[i] == 1)[0][0]:
        checkMatch2[i] = True

In [216]:
n - sum(checkMatch2)

3

### Define the function

In [237]:
def findAllocation(n, m, upperQ, lowerQ, a_ranks, c_score, typeConstr=1):
    model= Model("Assignment Model")
    model.Params.LogToConsole = 0
    
    X = {}
    for i in range(n):
        for j in range(m):
            X[i,j] = model.addVar(vtype= GRB.BINARY)
            
    if typeConstr == 1:        

        d = {}
        for i in range(n):
            for j in range(m):
                d[i,j] = model.addVar(lb=0.0, ub=float('inf'), vtype= GRB.CONTINUOUS)

        model.setObjective(quicksum(d[i,j] for i in range(n) for j in range(m)), GRB.MINIMIZE)

        #Constraint-1
        for i in range(n):
            model.addConstr(quicksum(X[i,j] for j in range(m)) <= 1)
        #Constraint-2
        for j in range(m):
            model.addConstr(quicksum(X[i,j] for i in range(n)) <= upperQ[j])

        #Constraint-3
        for j in range(m):
            model.addConstr(quicksum(X[i,j] for i in range(n)) >= lowerQ[j])        

        # for i in range(n):
        #     for j in range(m):
        #         model.addConstr(quicksum(X[i,k] for k in range(m) if (a_ranks[i][k] <= a_ranks[i][j]))*upperQ[j] \
        #                         + quicksum(X[h,j] for h in range(n) if c_score[h][j] > c_score[i][j]) >= upperQ[j])
    
        #Constraint-4
    
        for i in range(n):
            for j in range(m):
                model.addConstr(quicksum(X[i,k] for k in range(m) if (a_ranks[i][k] <= a_ranks[i][j]))*upperQ[j] \
                                + quicksum(X[h,j] for h in range(n) if c_score[h][j] >= c_score[i][j]) + d[i,j] \
                                >= upperQ[j])
                
        #Constraint-5
        for i in range(n):
            for j in range(m):
                model.addConstr(X[i,j] >= 0)
                
                
    else:        
        
        d = {}
        for i in range(n):
            for j in range(m):
                d[i,j] = model.addVar(vtype= GRB.BINARY)

        model.setObjective(quicksum(d[i,j] for i in range(n) for j in range(m)), GRB.MINIMIZE)

        #Constraint-1
        for i in range(n):
            model.addConstr(quicksum(X[i,j] for j in range(m)) <= 1)
        #Constraint-2
        for j in range(m):
            model.addConstr(quicksum(X[i,j] for i in range(n)) <= upperQ[j])

        #Constraint-3
        for j in range(m):
            model.addConstr(quicksum(X[i,j] for i in range(n)) >= lowerQ[j])        

        # for i in range(n):
        #     for j in range(m):
        #         model.addConstr(quicksum(X[i,k] for k in range(m) if (a_ranks[i][k] <= a_ranks[i][j]))*upperQ[j] \
        #                         + quicksum(X[h,j] for h in range(n) if c_score[h][j] > c_score[i][j]) >= upperQ[j])
       
        #Constraint-4
        for i in range(n):
            for j in range(m):
                model.addConstr(quicksum(X[i,k] for k in range(m) if (a_ranks[i][k] <= a_ranks[i][j]))*upperQ[j] \
                                + quicksum(X[h,j] for h in range(n) if c_score[h][j] >= c_score[i][j])\
                                + d[i,j]*upperQ[j] >= upperQ[j])        
        #Constraint-5
        for i in range(n):
            for j in range(m):
                model.addConstr(X[i,j] >= 0)
            
    model.optimize()
    
    res = np.array(model.X[:n*m]).reshape((n,m))
    
    return abs(res)

In [219]:
res3 = findAllocation(n, m, upperQ, lowerQ, a_ranks, c_score, 2)
res3

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.]])

### Testing

In [238]:
def checkMatch(n, m, res, a_ranks):
    a_ranksTable = np.array(a_ranks).reshape((n,m))
    
    checkMatch = [False for i in range(n)]
    
    for i in range(n):
        if np.where(res[i] == 1)[0][0] == np.where(a_ranksTable[i] == 1)[0][0]:
            checkMatch[i] = True

    return (n - sum(checkMatch))  

In [221]:
checkMatch(n, m, res3, a_ranks)

3

In [260]:
test_size = 40

n_array = np.sort([np.random.randint(25, 100) for i in range(test_size)])
m_array = np.sort([np.random.randint(3, 10) for i in range(test_size)])

In [261]:
#quotas
upperQ = [list(np.random.randint(int(n_array[i]/m_array[i])+10, \
                                 int(n_array[i]/m_array[i])+50, size=m_array[i])) for i in range(test_size)]

lowerQ = [list(np.random.randint(int(n_array[i]/m_array[i])-10, \
                                 int(n_array[i]/m_array[i]), size=m_array[i])) for i in range(test_size)]

In [262]:
#Students' ranks of companies
a_ranks = [[random.sample(list(np.arange(m_array[j])+1), m_array[j]) for i in range(n_array[j])] for j in range(test_size)]

#Companies' scores of students
c_score = [np.array([np.random.randint(1, 10, size=n_array[k]) for j in range(m_array[k])]).T for k in range(test_size)]

In [263]:
checkArray1 = []
checkArray2 = []

for i in range(test_size):
#     print(i)
    res1 = findAllocation(n_array[i], m_array[i], upperQ[i], lowerQ[i], a_ranks[i], c_score[i], 1)
    checkRes1 = checkMatch(n_array[i], m_array[i], res1, a_ranks[i])
    
    checkArray1.append(checkRes1)
    
    res2 = findAllocation(n_array[i], m_array[i], upperQ[i], lowerQ[i], a_ranks[i], c_score[i], 2)
    checkRes2 = checkMatch(n_array[i], m_array[i], res2, a_ranks[i])
    
    checkArray2.append(checkRes2)
    

In [264]:
checkArray1 == checkArray2

True